In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 13
TITLE = "23-24 January 2021"
SUBTITLE = "8 AM 24 January 2021"
sts = utc(2021, 1, 24, 5)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 40, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [15]:
cull = [548, 507, 'DAKI4', 'IA-KS-4', 'SACI4', 'GTHI4', 555, 'IA-AL-6']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
res = mp.postprocess(filename='210124.png')
mp.close()

     state  wfo  val       lon      lat                               geo  \
434     NY  BGM  1.9  -76.0400  42.1400    POINT (1941131.124 -30388.783)   
436     WI  MKX  1.9  -88.0200  42.9000    POINT (971644.186 -162066.466)   
437     WI  MKX  1.9  -87.7200  43.7500     POINT (981765.478 -64718.572)   
438     IA  DVN  1.9  -91.7500  41.5500    POINT (685409.802 -349137.776)   
439     WI  MKX  1.9  -89.3900  42.9200    POINT (861078.127 -175249.237)   
...    ...  ...  ...       ...      ...                               ...   
2134    TX  LUB  0.0 -100.0263  34.6888    POINT (-2414.392 -1145005.606)   
2135    TX  LUB  0.0 -101.2464  33.8761  POINT (-115605.600 -1234133.028)   
2136    TX  LUB  0.0 -100.9336  34.2675   POINT (-86167.235 -1191181.913)   
2137    TX  LUB  0.0 -100.9561  34.2319   POINT (-88283.702 -1195099.822)   
2138    TX  LUB  0.0 -102.7111  34.6425  POINT (-248980.505 -1146156.139)   

      used_for_analysis  nwsli  plotme source  
434                True    